# **Install Libraries**

In [20]:
!pip -q install langchain-huggingface transformers huggingface_hub bitsandbytes accelerate

In [21]:
!nvidia-smi

Thu Jan 30 11:21:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P0             26W /   70W |     631MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# **Import Libraries**

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

# **01 - HuggingFaceEndpoint integration of the free [Serverless Endpoints API](https://python.langchain.com/docs/integrations/llms/huggingface_endpoint/).**

In [23]:
prompt = PromptTemplate(
    input_variables = ['product'],
    template = "What is the good name for a company that makes {product}"
)

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=20,
    temperature=0.5,
    huggingfacehub_api_token=secret_value_0,
)
llm_chain = prompt | llm
print(llm_chain.invoke({"product": "cricket bats"}))

?

A good name for a company that makes cricket bats could be "Willow Wickets" or "Cricket Craftsmen." Other options could include "Batmaker's Guild," "Elegant Willows," or "Cricket Creations." These names convey a sense of tradition, craftsmanship, and the natural material used to make cricket bats, which is willow. Additionally, the names have a nice ring to them and are memorable.


# **02 - [Hugging Face Local Pipelines](https://python.langchain.com/docs/integrations/llms/huggingface_pipelines/)**

In [24]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, BitsAndBytesConfig

model_id = "google/flan-t5-large"

# Configure 8-bit quantization
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)

hf = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [25]:
hf

HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7c64ad35fac0>, model_id='google/flan-t5-large')

In [26]:
prompt = PromptTemplate(
    input_variables = ['product'],
    template = "What is the good name for a company that makes {product}"
)

llm_chain = prompt | hf
print(llm_chain.invoke({"product": "cricket bats"}))

scotts
